<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/main/workshop_sesjon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop sesjon 2: Analyse med BigQuery


Google's BigQuery er en serverless datawarehus løsning som støtter analyse av store datamengder basert på standard SQL plus noen ekstra funksjoner. I denne sesjonen skal vi gå gjennom noen av de fordelene man oppnå ved bruk av BigQuery til analyse av store datamengder versus det man kan få til lokalt/i en notebook eller kanskje en traditionell database.

Koden vi går gjennom kan både kjøres i denne noteboken og alternativt i BigQuery UI gjennom nettleseren.

### Autentisering med colab notebook

In [26]:
from google.colab import auth
auth.authenticate_user()

### Grunnlegende spørringer
BigQuery bruker standard SQL som default språk. Cellen under viser en enkelt spørring som bare returnerer alle rader fra vår trafikkdata tabell.

In [ ]:
%%bigquery --project <workshop_project> 
SELECT
 *
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 

Koper select statement inn til BigQuery UI og se på datamenge Query Validatoren angi. Hvor mye data innegår i spørringen?



BigQuery prises etter datamengenden som lagres eller analyseres. En spørring som analyserer mye data vil derfor koste mer.
Det finnes ulike måter å begrense datamengenden i en spørring. BigQuery behandler dataene kolonne-basert, dvs. jo flere kolonner man ta med i en spørring, desto mer kostnader oppstår.

Prøv å begrense spørringen ved a velge enkelte kolonner:

In [ ]:
%%bigquery --project <workshop_project> 
SELECT
  a.trpId
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 

Hvor mye data innegår i spørringen?

Bruk en enkel aggregasjon for å telle antall rader per  trafikktellepunkt ved bruk av `group by` 

In [ ]:
%%bigquery --project <workshop_project> 
SELECT
  a.trpId, COUNT(*) antall_rader
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 
group by 1

Datasettene i BigQuery er vanligvis denormalisert og det er vanlig å finne noen duplikator enten pga duplikasjon i underliggende datakilder eller som en resultat av avvik i ingestpipelinen. En enkel måte å finne disse duplikatene er ved å beregne total antall rader per en key som skal være unik, her f.eks. en kombinasjon av en tellepunkt ID og tidspunkt av måling:

In [ ]:
%%bigquery --project <workshop_project> 
SELECT
  a.trpId, a.ingest_time, COUNT(*) antall_rader
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 
group by 1,2
having antall_rader > 1

Hvor mange rader har minst 1 duplikat?

Fjern duplikatene! 

### Joins
BigQuery støtter alle vanlige typer joins som left/right join, inner join, full outer join, cross join.
Lag en left outer join, som knytter sammen trafikkdata med registreringspunkter lokasjonsinformasjon om registreringspunkter (GPS breddegrad og lengdegrad).

In [ ]:
%%bigquery --project <workshop_project> 
SELECT
  a.trpId,a.total.volumeNumbers.volume,b.location.coordinates.latLon
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` a
LEFT JOIN
  `saga-trafikkdata-stm-39pt.spike_eu.trafikkregistreringspunkter` b
ON
  a.trpId = b.id

### Views og materialized views

Views er en enkel måte å lagre en spørringen der resultatene oppdateres med endringer i underliggende datakilder. Bruk JOIN-spørringen og lagre resultatene som en view.


In [ ]:
%%bigquery --project <workshop_project> 
CREATE  VIEW <workshop_project>.<dataset>.<view_name> AS (
SELECT
  a.trpId,a.total.volumeNumbers.volume,b.location.coordinates.latLon
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` a
LEFT JOIN
  `saga-trafikkdata-stm-39pt.spike_eu.trafikkregistreringspunkter` b
ON
  a.trpId = b.id
)

Spør viewen i stedet for underliggende datakilder

In [ ]:
%%bigquery --project <workshop_project> 
SELECT * FROM <workshop_project>.<dataset>.<view_name>

Når man bruker en view som en del av analyser kan det være hensiktsmessig å lage en materialzed view. Materialized views lagrer en resultatene på en spørring som en tabell som automatisk oppdateres når underliggende tabellene oppdateres. Fordelen med en materialized view er at den i stor grad kan benytte seg av BigQuerys cache system slik at etterfølgende spørringen "koster" mye mindre data. En krav på materialiazed views er at de må inneholde et eller annet form av aggregasjon:

In [ ]:
%%bigquery --project <workshop_project> 
CREATE OR REPLACE MATERIALIZED VIEW `<workshop_project>.<dataset>.duplikat_view`
SELECT
  a.trpId, a.ingest_time, COUNT(*) antall_rader
FROM
  `saga-trafikkdata-stm-39pt.spike_eu.import_prod_mars` 
group by 1

### Tidspartisjonering

### Analytiske funksjoner

### GIS funksjoner

### User defined functions (UDFs)